# Import modules

In [27]:
import numpy as np
import pingouin as pg
import pandas as pd
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy


# Functions (helpers)

In [28]:
def read_csv(name, info=True, sep=','):
    df = pd.read_csv(f'../data/demographics/{name}.csv', sep=sep)
    # pd.set_option('display.max_columns', None)
    # pd.set_option('max_columns', None)
    if info:
        print_section('DATASET INFO')
        print('N rows=', df.size)
        print('N cols=', len(df.columns))
        print(df.head())
    return df

def print_section(title):
    print('\n')
    print( '*' + '-' * 135 + '*')
    print('* ' + title)
    print( '*' + '-' * 135 + '*')

def print_subsection(title):
    print('\n')
    print('***** ' + title + ' *****')
    print( '*' + '-' * 135 + '*')

In [29]:
# filenames = [
#    'interleaved_incomplete', 'block_complete',
#    'block_complete_simple',
#    'block_complete_mixed',  'block_complete_mixed_2s','block_complete_mixed_2s_amb_heuristic',
#    'block_complete_mixed_2s_amb_final', 'incentivize', 'evoutcome'

# ]
# dem = {}
# for f in filenames:
#     if f == 'block_complete_mixed_2s_amb_final':
#         dem[f] = pd.read_csv('../data/demographics/' + f + '.csv', sep=';')
#     else: dem[f] = read_csv(f)

# # d = dem["block_incomplete"][dem['block_incomplete']['status'] == 'APPROVED']
# # d
# df = {}
# ids = pd.read_csv('../sub_ids.csv')
# ids['prolific_id'] = 'nothing'
# for f in filenames:
#     df = pd.read_csv('../data/csv/' + f + '.csv', header=None)
#     # df.loc[-1] = [str(i) for i in range(len(df.columns))]  # adding a row
#     # df.index = df.index + 1  # shifting index
#     # df.sort_index(inplace=True) 
#     for i in ids[ids.exp_name==f]['id']:
#         try:
#             prolific_id = df[df[1]==i][2].unique()[0]
#         # break
#             ids.loc[ids.id==i, 'prolific_id'] = prolific_id
#         except:
#             try:
#                 prolific_id = df[df[1]==str(i)][2].unique()[0]
#                 ids.loc[ids.id==i, 'prolific_id'] = prolific_id
#             except:
#                 print(i, ' is missing...')
    
# # for i in ids['id']:
#     # df
# ids.to_csv('../sub_ids_with_prolific_id.csv', index=False)


In [30]:
# df = pd.read_csv('../sub_ids_with_prolific_id.csv')

# ids = df['prolific_id'].unique()
# exps = df['exp_name'].unique()
# df['age'] = np.nan
# df['gender'] = np.nan

# for f in exps:
#     sep = ','
#     if f == 'block_incomplete':
          # get prolific ids from demographics
#         ddf = pd.read_csv('../data/demographics/block_incomplete.csv')
#         ddf = ddf[ddf.status == 'APPROVED']
#         df.loc[df.exp_name=='block_incomplete', 'prolific_id'] = idx
#         ids = df['prolific_id'].unique()

#     if f == 'block_complete_mixed_2s_amb_final':
#         sep = ';'
#     print(f)
#     dem = pd.read_csv('../data/demographics/' + f + '.csv', sep=sep)
#     for i in ids:
#         try:
#             df.loc[df.prolific_id==i, 'gender'] = dem.loc[dem['participant_id'] == i, 'Sex'].values[0]
#             df.loc[df.prolific_id==i, 'age'] = dem.loc[dem['participant_id'] == i, 'age'].values[0]
#         except:
#             print(i, ' is missing...')
#             pass
# df.to_csv('../data/demographics/included_subjects.csv')

In [47]:
from cmath import exp


df = pd.read_csv('../data/demographics/included_subjects.csv')
#df[df['age'] == 'DATA_EXPIRED'] = np.random.choice('')
d = []
for e in df.exp_num.unique():
    gender = df[df.exp_num==e].gender.value_counts()
    age = df[df.exp_num==e].age.value_counts()
    exp_name = df[df.exp_num==e].exp_name.values[0]

    d.append(
        {   'exp_num': e,
            'exp_name': exp_name,
            'M': gender['Male'],
            '%M': (gender['Male']/sum(gender)).round(2),
            'F': gender['Female'],
            '%F': (gender['Female']/sum(gender)).round(2)
    })

pd.DataFrame(d)

,exp_num,exp_name,M,%M,F,%F
0,1,interleaved_incomplete,34,0.46,35,0.47
1,2,block_incomplete,26,0.37,39,0.55
2,3,block_complete,41,0.50,38,0.46
3,4,block_complete_simple,48,0.56,33,0.38
4,5,block_complete_mixed,38,0.54,31,0.44
5,6,block_complete_mixed_2s,32,0.48,32,0.48
6,7,block_complete_mixed_2s_amb_heuristic,32,0.47,36,0.53
7,8,block_complete_mixed_2s_amb_final,33,0.45,38,0.52
8,9,incentivize,47,0.64,25,0.34
9,10,evoutcome,42,0.52,37,0.46
